Alfie
====


**Alfie:Democratising RGBA Image Generation With No $$$**

 * Paper: https://arxiv.org/pdf/2408.14826v1

![Alfie Overview](../assets/alfie_overview.png)


### Instalation

```bash
git clone https://github.com/aimagelab/Alfie.git
cd Alfie

pip install torch torchvision
pip install -r requirements.txt
```

In [ ]:
import sys
sys.path.append("Alfie")
import json
import ast

import matplotlib.pyplot as plt
import torch
from transformers import VitMatteImageProcessor, VitMatteForImageMatting
from accelerate import PartialState
from accelerate.logging import get_logger
from accelerate.utils import set_seed

from alfie.generate import get_pipe
#from alfie.grabcut import grabcut, save_rgba
#from alfie.trimap import compute_trimap
#from alfie.utils import normalize_masks

torch.backends.cuda.matmul.allow_tf32 = True

SETTING_NAME = "centering-rgba-alfie"
settings_dict = parse_setting(SETTING_NAME)
print(settings_dict)
settings_dict = {
    k: v == "True"
    for k, v in settings_dict.items()
}

settings_dict

{'centering': 'True', 'use_neg_prompt': 'True', 'exclude_generic_nouns': 'True', 'cutout_model': 'grabcut', 'use_suffix': 'False'}


{'centering': True,
 'use_neg_prompt': True,
 'exclude_generic_nouns': True,
 'cutout_model': False,
 'use_suffix': False}

In [ ]:
distributed_state = PartialState()
device = distributed_state.device
print(device)

IMAGE_SIZE = 512
SCHEDULER = 'euler'  # choices=['euler', 'euler_ancestral']

pipe = get_pipe(
    image_size=IMAGE_SIZE,
    scheduler=SCHEDULER,
    device=device
)
pipe

cuda


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.78it/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00, 156.44it/s]


PixArtSigmaPipeline {
  "_class_name": "PixArtSigmaPipeline",
  "_diffusers_version": "0.29.2",
  "_name_or_path": "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS",
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5Tokenizer"
  ],
  "transformer": [
    "alfie.transformer_2d",
    "Transformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}